# Q2.1 Classic Machine Learning Methods

In [2]:
import pandas as pd
import numpy as np
from project_1.config import PROCESSED_DATA_DIR, PROJ_ROOT

SEED = 42

2025-03-21 23:49:57.597 | INFO     | project_1.config:<module>:11 - PROJ_ROOT path is: C:\Users\Menelao\Desktop\Health_care\project-1-ml4hc


In [3]:
# Load the data from Parquet files
sets_dict = {}
sets = ["a", "b", "c"]

for set_name in sets:
    directory = PROCESSED_DATA_DIR / f"set_{set_name}_final.parquet"
    temp_set = pd.read_parquet(directory)
    sets_dict[f"set_{set_name}"] = temp_set

# Assure the loading was correct
print(sets_dict["set_a"].shape)
sets_dict["set_a"].head(10)


(183416, 43)


,RecordID,Time,Gender,Height,Weight,Age,Albumin,Cholesterol,DiasABP,HCO3,...,ICUType,K,Lactate,MechVent,Urine,WBC,pH,SaO2,TroponinT,TroponinI
0,132539.0,2025-03-10 00:00:00,0.0,-0.953317,0.283445,-0.596332,1.004449,-0.069047,-0.739159,-1.269243,...,0.5,0.625,-0.583333,0.0,2.96,0.271429,-0.750,1.0,0.933333,-0.224490
1,132539.0,2025-03-10 01:00:00,0.0,-0.953317,0.283445,-0.596332,-0.232975,1.997003,0.232700,-2.924060,...,0.5,0.375,-0.166667,0.0,3.20,0.000000,0.750,-15.5,-0.177778,0.408163
2,132539.0,2025-03-10 02:00:00,0.0,-0.953317,0.283445,-0.596332,-1.132919,2.628972,2.440210,0.385573,...,0.5,0.625,2.000000,0.0,-0.40,2.185714,-0.250,1.0,0.333333,-0.285714
3,132539.0,2025-03-10 03:00:00,0.0,-0.953317,0.283445,-0.596332,-0.120482,-1.721888,1.169851,-0.441835,...,0.5,-1.000,-0.416667,0.0,0.72,2.185714,-0.250,0.5,0.333333,2.040816
4,132539.0,2025-03-10 04:00:00,0.0,-0.953317,0.283445,-0.596332,-0.120482,1.462261,-0.392067,-2.510356,...,0.5,0.125,0.000000,0.0,-0.16,-0.957143,0.375,1.0,0.200000,0.795918
5,132539.0,2025-03-10 05:00:00,0.0,-0.953317,0.283445,-0.596332,1.341928,1.146276,-0.235875,0.178721,...,0.5,-2.125,1.666667,0.0,-0.16,1.557143,-0.375,0.0,-0.244444,0.183673
6,132539.0,2025-03-10 06:00:00,0.0,-0.953317,0.283445,-0.596332,1.341928,-0.506564,-0.322648,-0.234983,...,0.5,0.625,0.833333,0.0,0.72,1.557143,-0.125,0.0,0.800000,-0.081633
7,132539.0,2025-03-10 08:00:00,0.0,-0.953317,0.283445,-0.596332,-1.357905,-0.117660,-0.010265,1.419833,...,0.5,-0.250,0.250000,0.0,0.32,-0.657143,-0.625,0.0,20.911111,8.408163
8,132539.0,2025-03-10 09:00:00,0.0,-0.953317,0.283445,-0.596332,-1.132919,0.344163,0.232700,-0.028131,...,0.5,-0.250,-0.416667,0.0,0.00,-0.657143,-0.250,0.0,-0.088889,-0.020408
9,132539.0,2025-03-10 10:00:00,0.0,-0.953317,0.283445,-0.596332,-1.470399,0.076791,0.302119,0.385573,...,0.5,-0.375,4.000000,0.0,0.16,-0.014286,-1.000,0.0,1.555556,0.040816


ICU TYpe da cavare

In [4]:

# Define file names
file_names = ["Outcomes-a.txt", "Outcomes-b.txt", "Outcomes-c.txt"]

# Directory path
base_path = PROJ_ROOT / "data" / "data_1" / "predicting-mortality-of-icu-patients-the-physionet-computing-in-cardiology-challenge-2012-1.0.0"

# Read files into DataFrames containing all variables
outcomes_a, outcomes_b, outcomes_c = [pd.read_csv(base_path / name) for name in file_names]

# Extract only the "RecordID" and "In-hospital_death" column into separate DataFrames
death_a, death_b, death_c = [df[["RecordID", "In-hospital_death"]] for df in [outcomes_a, outcomes_b, outcomes_c]]
death_a.head()

#CHECK for missing values in the outcome data
print(death_a.isnull().sum())
print(death_b.isnull().sum())
print(death_c.isnull().sum())



RecordID             0
In-hospital_death    0
dtype: int64
RecordID             0
In-hospital_death    0
dtype: int64
RecordID             0
In-hospital_death    0
dtype: int64


In [5]:
# Check if each "In-hospital_death" column contains only 0 and 1
for name, df in zip(["a", "b", "c"], [death_a, death_b, death_c]):
    print(f"\nValue counts for death_{name}:\n")
    print(df["In-hospital_death"].value_counts())
    print("\nContains only 0 and 1:", df["In-hospital_death"].isin([0, 1]).all())



Value counts for death_a:

In-hospital_death
0    3446
1     554
Name: count, dtype: int64

Contains only 0 and 1: True

Value counts for death_b:

In-hospital_death
0    3432
1     568
Name: count, dtype: int64

Contains only 0 and 1: True

Value counts for death_c:

In-hospital_death
0    3415
1     585
Name: count, dtype: int64

Contains only 0 and 1: True


So 1 means he died. We are going to fit a classifier to predict death. 

### Initial setup for the classifier to work

In [6]:
# Define the sets you want to modify
sets = ["set_a", "set_b", "set_c"]

# Delete the column ICUType from each set
for set_name in sets:
    sets_dict[set_name].drop(columns=["ICUType"], inplace=True)

# Check if the column was deleted by printing all the columns of set_a
sets_dict["set_a"].columns

Index(['RecordID', 'Time', 'Gender', 'Height', 'Weight', 'Age', 'Albumin',
       'Cholesterol', 'DiasABP', 'HCO3', 'HCT', 'HR', 'Mg', 'MAP', 'Na',
       'NIDiasABP', 'NIMAP', 'NISysABP', 'SysABP', 'PaCO2', 'PaO2',
       'Platelets', 'RespRate', 'Temp', 'ALP', 'ALT', 'AST', 'BUN',
       'Bilirubin', 'Creatinine', 'FiO2', 'GCS', 'Glucose', 'K', 'Lactate',
       'MechVent', 'Urine', 'WBC', 'pH', 'SaO2', 'TroponinT', 'TroponinI'],
      dtype='object')

In [7]:
# Define training, validation and testing sets
train_set = sets_dict["set_a"]
val_set = sets_dict["set_b"]
test_set = sets_dict["set_c"]


In [8]:

#change class of RecordID to int32
train_set["RecordID"] = train_set["RecordID"].astype("int32")
val_set["RecordID"] = val_set["RecordID"].astype("int32")
test_set["RecordID"] = test_set["RecordID"].astype("int32")


In [9]:
# Merge the training, validation and testing sets with the corresponding death DataFrames
train_set = train_set.merge(death_a, on="RecordID")
val_set = val_set.merge(death_b, on="RecordID") 
test_set = test_set.merge(death_c, on="RecordID")

# Check if the merge was successful by printing the first 5 rows of the training set
train_set.head()

,RecordID,Time,Gender,Height,Weight,Age,Albumin,Cholesterol,DiasABP,HCO3,...,K,Lactate,MechVent,Urine,WBC,pH,SaO2,TroponinT,TroponinI,In-hospital_death
0,132539,2025-03-10 00:00:00,0.0,-0.953317,0.283445,-0.596332,1.004449,-0.069047,-0.739159,-1.269243,...,0.625,-0.583333,0.0,2.96,0.271429,-0.750,1.0,0.933333,-0.224490,0
1,132539,2025-03-10 01:00:00,0.0,-0.953317,0.283445,-0.596332,-0.232975,1.997003,0.232700,-2.924060,...,0.375,-0.166667,0.0,3.20,0.000000,0.750,-15.5,-0.177778,0.408163,0
2,132539,2025-03-10 02:00:00,0.0,-0.953317,0.283445,-0.596332,-1.132919,2.628972,2.440210,0.385573,...,0.625,2.000000,0.0,-0.40,2.185714,-0.250,1.0,0.333333,-0.285714,0
3,132539,2025-03-10 03:00:00,0.0,-0.953317,0.283445,-0.596332,-0.120482,-1.721888,1.169851,-0.441835,...,-1.000,-0.416667,0.0,0.72,2.185714,-0.250,0.5,0.333333,2.040816,0
4,132539,2025-03-10 04:00:00,0.0,-0.953317,0.283445,-0.596332,-0.120482,1.462261,-0.392067,-2.510356,...,0.125,0.000000,0.0,-0.16,-0.957143,0.375,1.0,0.200000,0.795918,0


In [10]:
# Define aggregation rules
aggregation_rules = {
    "Age": "last",
    "Gender": "last",
    "Height": "last",
    "Albumin": "last",
    "ALP": "last",
    "ALT": "last",
    "AST": "last",
    "Bilirubin": "last",
    "BUN": "last",
    "Cholesterol": "last",
    "Creatinine": "last",
    "DiasABP": "mean",
    "FiO2": "mean",
    "GCS": "min",
    "Glucose": "mean",
    "HCO3": "last",
    "HCT": "last",
    "HR": "mean",
    "K": "last",
    "Lactate": "max",
    "Mg": "last",
    "MAP": "mean",
    "MechVent": "last",
    "Na": "last",
    "NIDiasABP": "mean",
    "NIMAP": "mean",
    "NISysABP": "mean",
    "PaCO2": "last",
    "PaO2": "mean",
    "pH": "last",
    "Platelets": "last",
    "RespRate": "mean",
    "SaO2": "mean",
    "SysABP": "mean",
    "Temp": "max",
    "TroponinI": "max",
    "TroponinT": "max",
    "Urine": "sum",
    "WBC": "last",
    "Weight": "last",
    "In-hospital_death": "max"  # If any 1 exists for a patient, return 1
}

# Perform aggregation
train_aggregated = train_set.groupby("RecordID").agg(aggregation_rules).reset_index()
val_aggregated = val_set.groupby("RecordID").agg(aggregation_rules).reset_index()
test_aggregated = test_set.groupby("RecordID").agg(aggregation_rules).reset_index()


# Display the processed dataset
train_aggregated.head()

,RecordID,Age,Gender,Height,Albumin,ALP,ALT,AST,Bilirubin,BUN,...,RespRate,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight,In-hospital_death
0,132539,-0.596332,0.0,-0.953317,-0.120482,0.372549,-0.263158,0.289474,0.222222,-0.55,...,-0.328319,-0.536458,0.158650,1.448365,8.408163,20.911111,38.320000,-0.300000,0.283445,0
1,132540,0.667051,1.0,0.615786,-0.120482,0.372549,-0.263158,0.289474,0.222222,0.10,...,-0.011361,-0.428571,-0.151136,1.096722,8.408163,20.911111,30.265333,0.257143,-0.232912,0
2,132541,-1.170597,0.0,-0.432085,-1.020426,0.431373,0.771930,0.771930,2.333333,-0.80,...,0.004531,-0.885417,0.175950,2.386078,8.408163,20.911111,24.688000,-0.757143,-1.078178,0
3,132543,0.207639,1.0,1.156482,1.341928,0.431373,-0.333333,-0.535088,-0.555556,-0.45,...,-0.746235,-0.525510,0.173347,0.041795,8.408163,20.911111,141.272000,-0.514286,0.143735,0
4,132545,1.356169,0.0,-1.117689,0.104504,0.372549,-0.263158,0.289474,0.222222,0.30,...,-0.051654,-0.239362,0.154832,0.979508,8.408163,20.911111,-2.032000,-0.957143,-0.835986,0


# Model 1 - Logistic Regression

In [11]:
# Separate Predictors (X) and Target (y)
X_train = train_aggregated.drop(columns=["RecordID", "In-hospital_death"])
y_train = train_aggregated["In-hospital_death"]

X_val = val_aggregated.drop(columns=["RecordID", "In-hospital_death"])
y_val = val_aggregated["In-hospital_death"]

X_test = test_aggregated.drop(columns=["RecordID", "In-hospital_death"])
y_test = test_aggregated["In-hospital_death"]

# Visualize the shape of the datasets
print(X_train.shape, y_train.shape)


(4000, 40) (4000,)


In [12]:
#El bondiano fa una logistica daje
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score

# --- Assume these DataFrames are already defined ---
# train_df = pd.read_csv("train.csv")
# valid_df = pd.read_csv("validate.csv")
# test_df  = pd.read_csv("test.csv")

# Separate features and target for each set.
# Adjust the column name "target" to your actual target column name.

# Create and train the Logistic Regression classifier.
clf = LogisticRegression(random_state=SEED, max_iter=1000)
clf.fit(X_train, y_train)

# Optionally, evaluate on the validation set.
y_valid_proba = clf.predict_proba(X_val)[:, 1]  # probability for the positive class
roc_auc_valid = roc_auc_score(y_val, y_valid_proba)
auprc_valid = average_precision_score(y_val, y_valid_proba)
print(f"Validation ROC AUC: {roc_auc_valid:.3f}, AUPRC: {auprc_valid:.3f}")

# Evaluate on the test set.
y_test_proba = clf.predict_proba(X_test)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Validation ROC AUC: 0.780, AUPRC: 0.397
Test ROC AUC: 0.769, AUPRC: 0.386


# Model 2 - Random Forest

In [13]:
# Use Random Forest to predict the target
from sklearn.ensemble import RandomForestClassifier

# Create and train the Random Forest classifier.
clf = RandomForestClassifier(random_state=SEED)
clf.fit(X_train, y_train)

# Optionally, evaluate on the validation set.
y_valid_proba = clf.predict_proba(X_val)[:, 1]  # probability for the positive class
roc_auc_valid = roc_auc_score(y_val, y_valid_proba)
auprc_valid = average_precision_score(y_val, y_valid_proba)
print(f"Validation ROC AUC: {roc_auc_valid:.3f}, AUPRC: {auprc_valid:.3f}")

# Evaluate on the test set.
y_test_proba = clf.predict_proba(X_test)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Validation ROC AUC: 0.779, AUPRC: 0.421
Test ROC AUC: 0.790, AUPRC: 0.424


## A little comment on the results:

AUC Score is pretty good. However, AUPRC is not that high. A problem that could arise is class imbalance. In fact, we have two classes, where the size of the 0 class is 6x the size of the 1 class. However, all three sets share the same problem, so it should not be a big problem. Another possible implementation could be using a dimensionality reduction technique, or some feature engineering.

# Q2.1 - Feature Engineering

We could use solutions such as *_Feature Lagging_*, *_Temporal Differences_* or *_Rolling Statistics_* to better capture the time-series trends.

### Attempt 2 - Signal processing-based features

Use tsfresh to extract a large amount of features. 

*Note* the use of MinimalFCParameters, to reduce the number of variables inspected (and so the high computational cost) 

Also tryied to substitute w/ EfficientFCParameters but still the run time is too much.

Maybe we should try to use the student cluster but we'll see



In [14]:
from tsfresh import extract_features, select_features
from tsfresh.feature_extraction.settings import MinimalFCParameters
from tsfresh.utilities.dataframe_functions import impute

# Define the column names
id_column = "RecordID"
time_column = "Time"
target_column = "In-hospital_death"

# Step 1: Extract all features from train_set using MinimalFCParameters
X_train_extracted = extract_features(
    train_set.drop(columns=[target_column]),
    column_id=id_column,
    column_sort=time_column,
    default_fc_parameters=MinimalFCParameters()  # Use minimal features
)

# Impute missing values (tsfresh requires no NaNs)
X_train_extracted = impute(X_train_extracted)

# Step 2: Select relevant features based on the target variable
y_train = train_set.groupby(id_column)[target_column].max()
X_train_relevant = select_features(X_train_extracted, y_train)

# Step 3: Store the relevant feature names
relevant_feature_names = X_train_relevant.columns

# Step 4: Extract the same features for val_set
X_val_extracted = extract_features(
    val_set.drop(columns=[target_column]),
    column_id=id_column,
    column_sort=time_column,
    default_fc_parameters=MinimalFCParameters()  # Use the same settings
)
X_val_extracted = impute(X_val_extracted)  # Impute missing values
X_val_relevant = X_val_extracted[relevant_feature_names]  # Keep only relevant features

# Step 5: Extract the same features for test_set
X_test_extracted = extract_features(
    test_set.drop(columns=[target_column]),
    column_id=id_column,
    column_sort=time_column,
    default_fc_parameters=MinimalFCParameters()  # Use the same settings
)
X_test_extracted = impute(X_test_extracted)  # Impute missing values
X_test_relevant = X_test_extracted[relevant_feature_names]  # Keep only relevant features

# Now X_train_relevant, X_val_relevant, and X_test_relevant have the same features
print("Train set relevant features:", X_train_relevant.shape)
print("Validation set relevant features:", X_val_relevant.shape)
print("Test set relevant features:", X_test_relevant.shape)

Feature Extraction: 100%|██████████| 20/20 [00:35<00:00,  1.76s/it]


Train set relevant features: (4000, 215)
Validation set relevant features: (4000, 215)
Test set relevant features: (4000, 215)


In [15]:
# For train_set
y_train = train_set.groupby(id_column)[target_column].max()

# For val_set
y_val = val_set.groupby(id_column)[target_column].max()

# For test_set
y_test = test_set.groupby(id_column)[target_column].max()


### Evaluation with Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler

# --- Scale the features ---
scaler = StandardScaler()

# Fit the scaler to the training data and transform
X_train_scaled = scaler.fit_transform(X_train_relevant)

# Transform the validation and test sets using the same scaler
X_val_scaled = scaler.transform(X_val_relevant)
X_test_scaled = scaler.transform(X_test_relevant)

# --- Create and train the Logistic Regression classifier ---
clf = LogisticRegression(random_state=42, max_iter=5000)  # Increase iterations to 5000
clf.fit(X_train_scaled, y_train)

# --- Evaluate on the validation set ---
y_val_proba = clf.predict_proba(X_val_scaled)[:, 1]  # Probability for the positive class
roc_auc_val = roc_auc_score(y_val, y_val_proba)
auprc_val = average_precision_score(y_val, y_val_proba)
print(f"Validation ROC AUC: {roc_auc_val:.3f}, AUPRC: {auprc_val:.3f}")

# --- Evaluate on the test set ---
y_test_proba = clf.predict_proba(X_test_scaled)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")


Validation ROC AUC: 0.821, AUPRC: 0.438
Test ROC AUC: 0.817, AUPRC: 0.469


Since there is a total of 215 features exctracted, we'll try with some regularization.

### Evaluation with Logistic Regression (l2-reg)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# --- Scale the features ---
scaler = StandardScaler()

# Fit the scaler to the training data and transform
X_train_scaled = scaler.fit_transform(X_train_relevant)

# Transform the validation and test sets using the same scaler
X_val_scaled = scaler.transform(X_val_relevant)
X_test_scaled = scaler.transform(X_test_relevant)

# --- Define the Logistic Regression model ---
# Use L2 regularization (default) and set up the solver
log_reg = LogisticRegression(random_state=42, max_iter=5000, penalty='l2', solver='liblinear')

# --- Set up the grid search for hyperparameter tuning ---
# Define the range of C values to test
param_grid = {'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}

# Use ROC AUC as the scoring metric for grid search
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, scoring='roc_auc', cv=5, verbose=1)

# --- Perform grid search on the training data ---
grid_search.fit(X_train_scaled, y_train)

# --- Get the best model and its parameters ---
best_clf = grid_search.best_estimator_
best_C = grid_search.best_params_['C']
print(f"Best regularization parameter (C): {best_C}")

# --- Evaluate on the validation set using the best model ---
y_val_proba = best_clf.predict_proba(X_val_scaled)[:, 1]  # Probability for the positive class
roc_auc_val = roc_auc_score(y_val, y_val_proba)
auprc_val = average_precision_score(y_val, y_val_proba)
print(f"Validation ROC AUC: {roc_auc_val:.3f}, AUPRC: {auprc_val:.3f}")

# --- Evaluate on the test set using the best model ---
y_test_proba = best_clf.predict_proba(X_test_scaled)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best regularization parameter (C): 0.01
Validation ROC AUC: 0.829, AUPRC: 0.458
Test ROC AUC: 0.827, AUPRC: 0.487


### Evaluation with Logistic Regression (l1- reg)

Since Lasso convergence is slower, we improved it by using the saga solver, increasing the iterations, adjusting the tolerance and also doing the grid search with every processor (n_jobs=-1)

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# --- Scale the features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_relevant)
X_val_scaled = scaler.transform(X_val_relevant)
X_test_scaled = scaler.transform(X_test_relevant)

# --- Define the Logistic Regression model with L1 regularization ---
log_reg = LogisticRegression(
    penalty='l1', 
    solver='saga', 
    max_iter=5000,  # Increase max_iter
    tol=1e-3,       # Adjust tolerance
    warm_start=True, # Enable warm start
    random_state=42
)
# --- Set up the grid search for hyperparameter tuning ---
param_grid = {'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, scoring='roc_auc', cv=5, n_jobs=-1, verbose=1)

# --- Perform grid search on the training data ---
grid_search.fit(X_train_scaled, y_train)

# --- Get the best model and its parameters ---
best_clf = grid_search.best_estimator_
best_C = grid_search.best_params_['C']
print(f"Best regularization parameter (C): {best_C}")

# --- Evaluate on the validation set using the best model ---
y_val_proba = best_clf.predict_proba(X_val_scaled)[:, 1]
roc_auc_val = roc_auc_score(y_val, y_val_proba)
auprc_val = average_precision_score(y_val, y_val_proba)
print(f"Validation ROC AUC: {roc_auc_val:.3f}, AUPRC: {auprc_val:.3f}")

# --- Evaluate on the test set using the best model ---
y_test_proba = best_clf.predict_proba(X_test_scaled)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best regularization parameter (C): 0.1
Validation ROC AUC: 0.834, AUPRC: 0.472
Test ROC AUC: 0.832, AUPRC: 0.491


### Evaluation with Random Forest

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score, average_precision_score
import numpy as np

# --- Define the Random Forest Classifier ---
rf = RandomForestClassifier(random_state=42, n_jobs=-1)  # Use all CPU cores

# --- Set up the randomized search for hyperparameter tuning ---
param_dist = {
    'n_estimators': [100, 200],  # Number of trees
    'max_depth': [10, 20],       # Maximum depth of each tree
    'min_samples_split': [2, 5], # Minimum samples to split a node
    'min_samples_leaf': [1, 2],  # Minimum samples at each leaf node
    'max_features': ['sqrt', 'log2']  # Number of features to consider at each split
}

# Use RandomizedSearchCV with a fixed number of iterations
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=10,  # Number of hyperparameter combinations to try
    scoring='roc_auc',
    cv=5,
    n_jobs=-1,  # Use all CPU cores
    verbose=1,
    random_state=42
)

# --- Perform randomized search on the training data ---
random_search.fit(X_train_relevant, y_train)

# --- Get the best model and its parameters ---
best_clf = random_search.best_estimator_
best_params = random_search.best_params_
print(f"Best hyperparameters: {best_params}")

# --- Evaluate on the validation set using the best model ---
y_val_proba = best_clf.predict_proba(X_val_relevant)[:, 1]  # Probability for the positive class
roc_auc_val = roc_auc_score(y_val, y_val_proba)
auprc_val = average_precision_score(y_val, y_val_proba)
print(f"Validation ROC AUC: {roc_auc_val:.3f}, AUPRC: {auprc_val:.3f}")

# --- Evaluate on the test set using the best model ---
y_test_proba = best_clf.predict_proba(X_test_relevant)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best hyperparameters: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 20}
Validation ROC AUC: 0.825, AUPRC: 0.450
Test ROC AUC: 0.825, AUPRC: 0.484


Without hyperparameter tuning:

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score

# --- Definisci il modello Random Forest con parametri predefiniti ---
rf = RandomForestClassifier(random_state=42, n_jobs=-1)  # Usa tutti i core della CPU

# --- Addestra il modello sui dati di training ---
rf.fit(X_train_relevant, y_train)

# --- Valuta sul validation set ---
y_val_proba = rf.predict_proba(X_val_relevant)[:, 1]  # Probabilità per la classe positiva
roc_auc_val = roc_auc_score(y_val, y_val_proba)
auprc_val = average_precision_score(y_val, y_val_proba)
print(f"Validation ROC AUC: {roc_auc_val:.3f}, AUPRC: {auprc_val:.3f}")

# --- Valuta sul test set ---
y_test_proba = rf.predict_proba(X_test_relevant)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Validation ROC AUC: 0.799, AUPRC: 0.395
Test ROC AUC: 0.811, AUPRC: 0.452
